In [1]:
from pathlib import Path
from rdkit import Chem
from iotbx.data_manager import DataManager
import numpy as np


In [2]:
import sys
sys.path.append("../PhenixML/")

# Old fragments

In [16]:
def build_fragment_heterograph(atom_graph,atom_idxs,connect_idxs,frag_idxs):
    # build fragment graph

    frag_name = "fragment"



    # initialize empty dictionary
    hg = {}
    hg[("atom", "atom_connects_atom", "atom")] = connect_idxs

    idxs_renamed = {}
    idxs_renamed["atom"] = idxs["n1"]
    idxs_renamed[frag_name] = frag_idxs

    idxs["atom"] = atom_idxs
    idxs["connect"] = connect_idxs
    idxs[frag_name] = frag_idxs

    terms = ["atom",frag_name] # debug
    levels = terms
    
    
    idxs_to_ordering = {}
    for term in terms:
      idxs_to_ordering[term] = {
          tuple(subgraph_idxs): ordering
          for (ordering, subgraph_idxs) in enumerate(list(idxs[term]))
      }

    # this needs rewriting. Current is based on espaloma
    for small_idx in range(1, len(terms)+1):
      for big_idx in range(small_idx + 1, len(terms)+1):
          for pos_idx in range(frag_idxs.shape[1]):
              hg[
                  (
                      "atom",
                      "atom_as_%s_in_%s" % (pos_idx,frag_name),
                      frag_name
                  )
              ] = np.stack(
                  [
                      np.array(
                          [
                              idxs_to_ordering["atom"][tuple(x)]
                              for x in idxs[frag_name][
                                  :, pos_idx : pos_idx + small_idx
                              ]
                          ]
                      ),
                      np.arange(idxs[frag_name].shape[0]),
                  ],
                  axis=1,
              )
              # #comment out has relationships. I think this is ok since we are only using the higher level nodes for readout
              # hg[
              #     (
              #         "n%s" % big_idx,
              #         "n%s_has_%s_n%s" % (big_idx, pos_idx, small_idx),
              #         "n%s" % small_idx,
              #     )
              # ] = np.stack(
              #     [
              #         np.arange(idxs["n%s" % big_idx].shape[0]),
              #         np.array(
              #             [
              #                 idxs_to_ordering["n%s" % small_idx][tuple(x)]
              #                 for x in idxs["n%s" % big_idx][
              #                     :, pos_idx : pos_idx + small_idx
              #                 ]
              #             ]
              #         ),
              #     ],
              #     axis=1,
              # )
    import dgl
    hg = dgl.heterograph({key: list(value) for key, value in hg.items()})

    hg.nodes["atom"].data["h0"] = atom_graph.ndata["h0"].type(torch.get_default_dtype())
    # set the n1 nodes to have the features from the atoms in the homograph

    # include indices in the idxs to nodes in the homograph
    for term in levels:
        hg.nodes[term].data["idxs"] = torch.tensor(idxs[term])

    return hg

In [17]:
idxs = get_indices_from_mol(mol)
atom_idxs = idxs["n1"]
connect_idxs = idxs["n2"]
frag_idxs = idxs["n3"]

In [18]:
hg = build_fragment_heterograph(g,atom_idxs,connect_idxs,frag_idxs)

In [19]:
from phenixml.models.message_passing import MessagePassing
from phenixml.models.readout import JanossyReadout

In [20]:
#from molgraph.esp_fragments.sequential import _Sequential, EspSequential
#from molgraph.esp_fragments.pooling import EspJanossyPooling

## My version

In [21]:
n_units = 128
n_layers = 3
fragment_size = 3
config = []
for n_layer in range(n_layers):
  config+=[n_units,"relu"]
  
  
sc = lambda in_feats,out_feats: dgl.nn.pytorch.conv.sageconv.SAGEConv(in_feats,out_feats,"mean",bias=True) # dgl saveconv layer

representation = MessagePassing(
    feature_units=hg.nodes["atom"].data["h0"].shape[1],
    layer=sc, # use SAGEConv implementation in DGL
    #layer=esp.nn.dgl_legacy.gn("SAGEConv"),
    config=config,
) 

In [22]:
n_units = 128
n_layers = 2
config = []
for n_layer in range(n_layers):
  config+=[n_units,"relu"]
  
readout = JanossyReadout(
    in_features=n_units, config=config,
    out_features=["feat1","feat2"],
    fragment_size = 3,
)



## Esp version


In [33]:
from molgraph.esp_fragments.sequential import _Sequential, EspSequential
from molgraph.esp_fragments.pooling import EspJanossyPooling
from molgraph.build_graphs import *
g = build_homograph(mol)
hg = build_heterograph_from_homo_mol(g,mol,levels=["n1","n2","n3"])


In [35]:

n_units = 128
n_layers = 3
config = []
for n_layer in range(n_layers):
  config+=[n_units,"relu"]
  
  
sc = lambda in_feats,out_feats: dgl.nn.pytorch.conv.sageconv.SAGEConv(in_feats,out_feats,"mean",bias=True) # dgl saveconv layer

representation = EspSequential(
    feature_units=g.ndata["h0"].shape[1],
    layer=sc, # use SAGEConv implementation in DGL
    #layer=esp.nn.dgl_legacy.gn("SAGEConv"),
    config=config,
) 
n_units = 128
n_layers = 2
config = []
for n_layer in range(n_layers):
  config+=[n_units,"relu"]
  
readout = EspJanossyPooling(
    in_features=n_units, config=config,
    out_features={              
        #1: {"xyz": 3},
        3: {"eq": 1},
        #4: {"cmplx": 2},
        #4: {"tor": 1},
        },
)

In [23]:
%time g = representation(hg)

CPU times: user 2.08 s, sys: 148 ms, total: 2.23 s
Wall time: 81.8 ms


In [25]:
hg.etypes

['atom_as_0_in_fragment',
 'atom_as_1_in_fragment',
 'atom_as_2_in_fragment',
 'atom_connects_atom']

In [24]:
%time g = readout(hg)

CPU times: user 1.05 s, sys: 42.3 ms, total: 1.09 s
Wall time: 21 ms


In [ ]:
#angle_fragments = list(itertools.chain.from_iterable([fragmenter(container) for container in containers]))